In [8]:
from langgraph.graph import StateGraph, START,END 
from langchain_google_genai import ChatGoogleGenerativeAI 
import os 
from typing import TypedDict, Annotated, Literal 
from langchain_core.messages import BaseMessage, HumanMessage 
from dotenv import load_dotenv 
from pydantic import BaseModel,Field 
from langgraph.checkpoint.memory import InMemorySaver

In [2]:
load_dotenv()

True

In [3]:
os.environ['GOOGLE_API_KEY'] = os.getenv('gemini_api_key') 
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [4]:
class JokeState(TypedDict):
    topic: str
    joke: str
    explanination: str 
    

In [5]:
def generate_joke(state:JokeState):
    prompt = f'generate a joke on the topic {state["topic"]}'
    response = model.invoke(prompt).content 
    return {"joke":response}

In [7]:
def get_explaination(state:JokeState):
    prompt = f'generate  explaination for the joke {state["joke"]}' 
    response = model.invoke(prompt).content 
    return {"explanation":response}

In [9]:
graph = StateGraph(JokeState) 

graph.add_node("generate_joke",generate_joke)
graph.add_node("generate_explaination",get_explaination) 

graph.add_edge(START,'generate_joke')
graph.add_edge('generate_joke','generate_explaination')
graph.add_edge('generate_explaination',END) 


checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer) 





In [11]:
config1 = {"configurable":{"thread_id":"1"}}
workflow.invoke({"topic":"pizza"},config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza get a promotion?\n\nBecause it had a lot of **crust**!'}

In [13]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza get a promotion?\n\nBecause it had a lot of **crust**!'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0bd093-b248-6e93-8006-85e9c7316a15'}}, metadata={'source': 'loop', 'step': 6, 'parents': {}}, created_at='2025-11-09T01:12:59.309224+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0bd093-78f2-6deb-8005-2cd39578a2b9'}}, tasks=(), interrupts=())